In [ ]:
from functools import partial
import jax
import jax.numpy as np
from flax import linen as nn
from jax.nn.initializers import lecun_normal, normal
from jax.numpy.linalg import eigh, inv, matrix_power
from jax.scipy.signal import convolve

from S4 import *
rng = jax.random.PRNGKey(1)

In [ ]:
S4Layer = cloneLayer(S4Layer)

In [ ]:
model = S4Layer(N=512, l_max=6)